#Suumoスクレイピングで情報取得

東京都千代田区のエリアにおいて下記の必要最低限を含む情報を取得し、データ抽出し格納、csvファイルで保存する。

◇物件情報

・物件名

・住所


◇部屋情報

・間取り

・家賃

・階＃＃

In [1]:
#ライブラリのインポート
import requests
from bs4 import BeautifulSoup
from time import sleep
import pandas as pd

#アクセス先のURLの設定
REQUEST_URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page={}'

#空のリストを作成
d_list = []

#複数ページをforループにて取得（33ページ目まで）
for i in range(1,34):
    print("d_listの大きさ",len(d_list))

    TARGET_URL = REQUEST_URL.format(i)

    print(TARGET_URL)

    #requestにより設定したURLにアクセス
    res = requests.get(TARGET_URL)

    #1秒間のsleepをはさむ
    sleep(1)

    #取得した情報をbeautiful Soupで解析
    soup = BeautifulSoup(res.text,"html.parser")

        #すべての物件情報を取得
    contents = soup.find_all('div',class_="cassetteitem")

    #各物件情報をforループで取得
    for content in contents:
        #物件情報と部屋情報をそれぞれ取得
        detail = content.find('div',class_="cassetteitem-detail")
        item = content.find('div',class_="cassetteitem-item")

        #物件情報から物件名、住所を取得
        title = detail.find('div',class_='cassetteitem_content-title').text
        address = detail.find('li',class_='cassetteitem_detail-col1').text

        #部屋情報から間取り、家賃、階を取得するための情報を取得
        tr_tags = item.find_all("tr", class_="js-cassette_link")

        #部屋情報は複数あり、各部屋情報を取得
        for tr_tag in tr_tags:
            #各部屋情報から取得したい情報をそれぞれの変数に格納
            floor,price,first_fee,capacity = tr_tag.find_all("td")[2:6]

            #priceの中身を賃料(fee)、管理費(management_fee)でそれぞれ分けて格納
            fee, management_fee = price.find_all("li")

            #first_feeの中身を敷金(deposit)、礼金(gratuity)でそれぞれ分けて格納
            deposit, gratuity = first_fee.find_all("li")

            #capacityの中身を間取り(madori)、専有面積(menseki)でそれぞれ分けて格納
            madori, menseki = capacity.find_all("li")

            #変数dにこれまで取得した9項目を格納する
            d = {
                "title":title,
                "address":address,
                "floor":floor.text,
                "fee":fee.text,
                "management_fee":management_fee.text,
                "deposit":deposit.text,
                "gratuity":gratuity.text,
                "madori":madori.text,
                "menseki":menseki.text
            }

            #取得した辞書をd_listに格納する
            d_list.append(d)

d_listの大きさ 0
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=1
d_listの大きさ 195
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=2
d_listの大きさ 307
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=3
d_listの大きさ 442
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=&srch_navi=1&page=4
d_listの大きさ 565
https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=13&sc=13101&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr

In [2]:
#取得した情報をデータフレームに格納し、csvに変換して保存
df = pd.DataFrame(d_list)
df.to_csv("SUUMO_scraping_ver2.csv",index=None, encoding="utf-8-sig")